In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
from astropy.io import fits
import astropy.units as u
from astropy.cosmology import Planck18 as cosmo, z_at_value
import coop_post_processing as cpp
from scipy import ndimage
from astropy import constants
import coop_setup_funcs as csf
import warnings
warnings.filterwarnings('ignore')
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=False)
# set plot params
import matplotlib as mpl
import scipy as sp
mpl.font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 3}
plt.rcParams.update({'font.size': 14})
mpl.rcParams['figure.figsize'] = [8.0, 6.0]
import healpy as hp
import stats as st

import importlib
importlib.reload(cpp)


# 1. Load a redshift bin for y, galaxy, and kappa
# 2. Check for all that the number of stacks per region is identical

In [ ]:
dbins = [[893, 1393], [1493, 1993], [2093, 2593], [2693,3193]]
for dbin in dbins:
    print(z_at_value(cosmo.comoving_distance, (dbin[0]-50)*u.Mpc),z_at_value(cosmo.comoving_distance, (dbin[1]+50)*u.Mpc))

zbins = ['0pt20_0pt36', '0pt36_0pt53', '0pt53_0pt72', '0pt72_0pt94'] # the z bins for number density maps
actpath  = "/mnt/scratch-lustre/mlokken/stacking/ACTxDES_paper2/"
buzzpath = "/mnt/scratch-lustre/mlokken/stacking/Buzzard_paper2/"
mask =  "/mnt/raid-cita/mlokken/data/masks/y3_gold_2.2.1_RING_joint_redmagic_v0.5.1_wide_maglim_v2.2_mask_hpx_4096.fits"

In [ ]:
nu_e_cuts = True
if nu_e_cuts:
    pt_selection_str = 'nugt2_egtpt3_'
else:
    pt_selection_str = ''
nreg = 24


In [90]:

mmax=2
b = 7.5
average_pks_per_dbin = []
binned_profiles_actdes = []
binned_errors_actdes = []
binned_profiles_buzz = []
binned_errors_buzz = []
binned_profiles_buzzbreak = []

pix_area_sq_arcmin = (hp.nside2pixarea(4096, degrees=True)*(u.deg)**2).to(u.arcmin**2).value
for col, dbin in enumerate(dbins):
    
    cl_dlow, cl_dhi = dbin[0], dbin[1]
    dlow   = cl_dlow-50
    dhi    = cl_dhi+50
    zlow   = z_at_value(cosmo.comoving_distance, cl_dlow*u.Mpc)
    zhi    = z_at_value(cosmo.comoving_distance, cl_dhi*u.Mpc)
    # get the galaxy conversions here
    
    mapstr = f'DES_maglim_z_{zbins[col]}'
    des, (zlow,zhi) = cpp.retrieve_stack_info(actpath, mapstr, pt_selection_str, dbin, scale=4/pix_area_sq_arcmin, remove_r30=False) # convert to galaxies / arcmin^2
   
    mapstr = f'ilc_SZ_deproj_cib_1.0_10.7_yy_4096_hpx'    
    act, (zlow,zhi) = cpp.retrieve_stack_info(actpath, mapstr, pt_selection_str, dbin, remove_r30=True)

    mapstr = f'kappa_bin4'    
    desk, (zlow,zhi) = cpp.retrieve_stack_info(actpath, mapstr, pt_selection_str, dbin, remove_r30=False)

    mapstr = f'Buzzard_maglim_z_{zbins[col]}'
    buzzg, (zlow,zhi) = cpp.retrieve_stack_info(buzzpath, mapstr, pt_selection_str, dbin, scale=4/pix_area_sq_arcmin, remove_r30=True) # convert to galaxies / arcmin^2
    
    
    mapstr = f'ymap_buzzard_fid_hpx'
    buzzy, (zlow,zhi) = cpp.retrieve_stack_info(buzzpath, mapstr, pt_selection_str, dbin, remove_r30=True)

    print(np.sort(des.Npks_splits))
    print(np.sort(act.Npks_splits))
    print(np.sort(desk.Npks_splits))
    
    # print(buzzg.Npks_splits)
    # print(buzzy.Npks_splits)
    

retrieving data from /mnt/scratch-lustre/mlokken/stacking/ACTxDES_paper2/DES_maglim_z_0pt20_0pt36_redmapper_lambdagt20_combined_893_1393Mpc_nugt2_egtpt3_20pt0_orientXYUP_75pct_maglim_843_1443Mpc_24reg_m0to5_profiles.pkl
retrieving data from /mnt/scratch-lustre/mlokken/stacking/ACTxDES_paper2/ilc_SZ_deproj_cib_1.0_10.7_yy_4096_hpx_redmapper_lambdagt20_combined_893_1393Mpc_nugt2_egtpt3_20pt0_orientXYUP_75pct_maglim_843_1443Mpc_24reg_m0to5_profiles.pkl
retrieving data from /mnt/scratch-lustre/mlokken/stacking/ACTxDES_paper2/kappa_bin4_redmapper_lambdagt20_combined_893_1393Mpc_nugt2_egtpt3_20pt0_orientXYUP_75pct_maglim_843_1443Mpc_24reg_m0to5_profiles.pkl
retrieving data from /mnt/scratch-lustre/mlokken/stacking/Buzzard_paper2/Buzzard_maglim_z_0pt20_0pt36_redmapper_lambdagt20_combined_893_1393Mpc_nugt2_egtpt3_20pt0_orientXYUP_75pct_maglim_843_1443Mpc_24reg_m0to5_profiles.pkl
retrieving data from /mnt/scratch-lustre/mlokken/stacking/Buzzard_paper2/ymap_buzzard_fid_hpx_redmapper_lambdagt20_c

In [87]:
np.sort(npks_list), np.sort(des.Npks_splits), des.


(array([ 1,  6,  7,  8, 10, 11, 11, 12, 15, 16, 19, 19, 20, 22, 25, 27, 31,
        31, 32, 33, 36, 43, 47, 48]),
 array([ 1,  6,  7,  8, 10, 11, 11, 12, 15, 16, 19, 19, 20, 22, 25, 27, 31,
        31, 32, 33, 36, 43, 47, 48]),
 array([15,  6, 12, 19, 20, 19, 48, 27, 10, 47, 43, 33, 25, 16, 11, 11, 22,
        31,  1, 36, 31,  8, 32,  7]))

In [ ]:
# test case 1
testcase = "redmapper_lambdagt20_1493_1593Mpc_nugt2_egtpt3_20pt0_orientXYUP_75pct_maglim_1443_1643Mpc_reg1"

In [76]:
dbin = [893, 1393]
npks_list = []
for reg in range(24):
    npks_reg = 0
    for d in range(dbin[0], dbin[1], 100):
        cl_dlow, cl_dhi = d, d+100
        dlow   = cl_dlow-50
        dhi    = cl_dhi+50

        testcase = f"redmapper_lambdagt20_{cl_dlow}_{cl_dhi}Mpc_nugt2_egtpt3_20pt0_orientXYUP_75pct_maglim_{dlow}_{dhi}Mpc_reg{reg}"
        path = f"/mnt/scratch-lustre/mlokken/stacking/ACTxDES_paper2/orient_by_maglim_75/stacks/{reg}"
        if os.path.isfile(f"{path}/{testcase}_pks.fits"):
            pksregfile = fits.open(f"{path}/{testcase}_pks.fits")
            dat = pksregfile[0].data
            pksregfile.close()
        
            # print(len(dat))
            npks_reg += len(dat)
    
            kapparegfile = fits.open(f"{path}/kappa_bin4_{testcase}_stk.fits")
            kdat = kapparegfile[0].data
            kapparegfile.close()
            galregfile = fits.open(f"{path}/DES_maglim_z_{zbins[0]}_{testcase}_stk.fits")
            gdat = galregfile[0].data
            galregfile.close()
            yregfile = fits.open(f"{path}/ilc_SZ_deproj_cib_1.0_10.7_yy_4096_hpx_{testcase}_stk.fits")
            ydat = yregfile[0].data
            yregfile.close()
    print(npks_reg)
    npks_list.append(npks_reg)

19
11
27
16
20
32
15
11
10
7
43
8
36
48
31
22
6
33
47
12
1
25
31
19


In [ ]:
npks

In [ ]:
pksregfile = fits.open(f"/mnt/scratch-lustre/mlokken/stacking/ACTxDES_paper2/orient_by_maglim_75/stacks/1/{testcase}_pks.fits")
dat = pksregfile[0].data
pksregfile.close()
print(len(dat))


In [ ]:
kapparegfile = fits.open(f"/mnt/scratch-lustre/mlokken/stacking/ACTxDES_paper2/orient_by_maglim_75/stacks/1/kappa_bin4_{testcase}_stk.fits")
dat = kapparegfile[0].data
kapparegfile.close()
plt.imshow(dat)

In [ ]:
galregfile = fits.open(f"/mnt/scratch-lustre/mlokken/stacking/ACTxDES_paper2/orient_by_maglim_75/stacks/1/DES_maglim_z_{zbins[1]}_{testcase}_stk.fits")
dat = galregfile[0].data
galregfile.close()
plt.imshow(dat)

In [ ]:
galregfile = fits.open(f"/mnt/scratch-lustre/mlokken/stacking/ACTxDES_paper2/orient_by_maglim_75/stacks/1/ilc_SZ_deproj_cib_1.0_10.7_yy_4096_hpx_{testcase}_stk.fits")
dat = galregfile[0].data
galregfile.close()
plt.imshow(dat)

In [ ]:
z_at_value(cosmo.comoving_distance, 1493*u.Mpc)

In [ ]:
ls /mnt/scratch-lustre/mlokken/stacking/ACTxDES_paper2/orient_by_maglim_75/stacks/14/DES_maglim_z_0pt20_0pt36_redmapper_lambdagt20_893_993Mpc_nugt2_egtpt3_20pt0_orientXYUP_75pct_maglim_843_1043Mpc_reg14_stk.txt